# MidTerm_Assignment

### Problem_Statement

You are given the reviews dataset. These are 194439 amazon reviews for cell phones and
accessories taken from https://jmcauley.ucsd.edu/data/amazon/ Use the “reviewText” and
“overall” fields from this file. The goal is to predict the rating given the review by modeling it as
a multi-class classification problem.

1. Take the first 70% dataset for train, next 10% for validation/development, and remaining
20% for test.
2. Traditional machine learning methods
a. Design some good linguistic features. You can start with basic TFIDF features.
Use these classifiers: J48 decision trees, SVMs with linear/RBF kernel, logistic
regression, xgboost, random forests and report accuracy on test set.


### Importing the Required Libraries

In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Download stopwords and tokenizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to C:\Users\Bhanu
[nltk_data]     prakash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Bhanu
[nltk_data]     prakash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Bhanu
[nltk_data]     prakash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Reading the data file

In [4]:
# Load the entire dataset
full_data = pd.read_json("D:/Studies/AI and Decesion Science 2/Mid_Term/dataset/Cell_Phones_and_Accessories_5.json", lines=True)

# Take a random sample of 5% of the data
sample_size = int(0.05 * len(full_data))
data = full_data.sample(n=sample_size, random_state=42)

In [5]:
data.head()

reviewerID        asin       reviewerName helpful  \
156187  A1P3ACSVXZ1JUM  B00C0ZKI9Q              LO127  [0, 0]   
102252  A2Z3IZDQUIDRR9  B008FQVO5G            yogitim  [8, 8]   
23146   A3EOVXI1VZIHUQ  B003Y74AZ2  Z. Freeman "Zach"  [1, 2]   
86461    AWU526GBZ64MH  B007ZL0TJ4               Bree  [0, 0]   
62407   A3F7F7QKQP2FKT  B005VNK7T0             Beanie  [0, 0]   

                                               reviewText  overall  \
156187  iBOLT xProDock Active Car Dock/Holder/Mount fo...        5   
102252  This pouch is everything that I was looking fo...        5   
23146   First of all - the case I had on my iPhone 4 p...        3   
86461   I ordered this case because I had the same cas...        5   
62407   Bought a few of these as gifts. They were a bi...        5   

                                       summary  unixReviewTime   reviewTime  
156187                          Fits perfectly      1372032000  06 24, 2013  
102252                            Great Pouch!      1361577600  02 23, 2013  
23146   An average iPhone case - nothing fancy      1296604800   02 2, 2011  
86461                               Great Case      1379635200  09 20, 2013  
62407                                     Cute      1371859200  06 22, 2013

## Preprocessing

In [6]:
# Preprocess text data (tokenization, lowercasing, lemmatization, stopword removal)
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words if word.isalpha() and word.lower() not in stop_words]
    return ' '.join(words)



In [7]:
# Apply text preprocessing to the 'reviewText' column
data['reviewText'] = data['reviewText'].apply(preprocess_text)

#### Remove punctuation and numbers using regular expressions

In [8]:
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data['reviewText'] = data['reviewText'].apply(lambda x: re.sub(r'\d+', '', x))

#### Remove words with low frequency

In [9]:
min_frequency = 5
word_frequency = data['reviewText'].str.split(expand=True).stack().value_counts()
low_freq_words = word_frequency[word_frequency <= min_frequency].index
data['reviewText'] = data['reviewText'].apply(lambda x: ' '.join(word for word in x.split() if word not in low_freq_words))


In [10]:
# Display the top 10 most frequent words
print(word_frequency.head(10))

phone      9762
case       8250
one        4994
like       3808
great      3369
screen     3273
battery    3205
use        3188
work       3083
would      3000
dtype: int64


#### Label encoding for the target variable 'overall'

In [11]:
label_encoder = LabelEncoder()
data['overall'] = label_encoder.fit_transform(data['overall'])

#### Take the first 70% for train, next 10% for validation, and remaining 20% for test

In [12]:
train_data, remaining_data = train_test_split(data, train_size=0.7, random_state=42)
valid_data, test_data = train_test_split(remaining_data, train_size=0.33, random_state=42)



In [13]:
# Extract 'reviewText' and 'overall' fields
x_train, y_train = train_data['reviewText'], train_data['overall']
x_valid, y_valid = valid_data['reviewText'], valid_data['overall']
x_test, y_test = test_data['reviewText'], test_data['overall']



### Creating TF-IDF vectorizer and transform the reviews into feature vectors

In [14]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
x_train_features = tfidf_vectorizer.fit_transform(x_train)
x_valid_features = tfidf_vectorizer.transform(x_valid)
x_test_features = tfidf_vectorizer.transform(x_test)

In [15]:
# Get the feature names (words) used in the TF-IDF matrix
feature_names = tfidf_vectorizer.get_feature_names_out()

# Print the first 120 feature names
print("Features:")
print(feature_names[:120])

Features:
['aa' 'ability' 'able' 'abroad' 'absolute' 'absolutely' 'absorb'
 'absorbing' 'abuse' 'ac' 'acase' 'accent' 'accept' 'acceptable' 'access'
 'accessed' 'accessibility' 'accessible' 'accessing' 'accessory'
 'accident' 'accidental' 'accidentally' 'accidently' 'accommodate'
 'according' 'account' 'accuracy' 'accurate' 'accurately' 'ace' 'achieve'
 'across' 'act' 'acting' 'action' 'activate' 'activated' 'activates'
 'activating' 'activation' 'active' 'activity' 'actual' 'actually' 'ad'
 'adapted' 'adapter' 'adaptor' 'add' 'added' 'adding' 'addition'
 'additional' 'additionally' 'address' 'adequate' 'adequately' 'adhere'
 'adhesive' 'adjust' 'adjustable' 'adjusted' 'adjusting' 'adjustment'
 'adjusts' 'admit' 'admittedly' 'adobe' 'adorable' 'adore' 'adult'
 'advance' 'advanced' 'advantage' 'advertise' 'advertised' 'advertising'
 'advice' 'advise' 'aesthetic' 'aesthetically' 'affect' 'affected'
 'afford' 'affordable' 'afraid' 'aftermarket' 'afternoon' 'age' 'ago'
 'agree' 'ahead' 'ai

# Model Building

### Decision Tree Classifier

In [16]:
# Decision Tree Classifier - Hyperparameter Tuning
dt_param_grid = {
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dt_classifier = DecisionTreeClassifier()
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=3, n_jobs=-1)
dt_grid_search.fit(x_train_features, y_train)
best_dt_classifier = dt_grid_search.best_estimator_
dt_valid_predictions = best_dt_classifier.predict(x_valid_features)
dt_test_predictions = best_dt_classifier.predict(x_test_features)


In [17]:
print("Decision Tree Classifier (Grid Search):")
print(classification_report(y_valid, dt_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, dt_valid_predictions))



Decision Tree Classifier (Grid Search):
              precision    recall  f1-score   support

           0     0.1765    0.0441    0.0706        68
           1     0.0000    0.0000    0.0000        49
           2     0.0000    0.0000    0.0000       105
           3     0.3628    0.2020    0.2595       203
           4     0.5844    0.8957    0.7074       537

    accuracy                         0.5457       962
   macro avg     0.2247    0.2284    0.2075       962
weighted avg     0.4153    0.5457    0.4546       962

Accuracy: 0.5457380457380457


In [18]:
# Print classification report and accuracy for the Decision Tree model on the test set
print("\nResults on the test set:")
print("\nDecision Tree Classifier:")
print(classification_report(y_test, dt_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, dt_test_predictions))


Results on the test set:

Decision Tree Classifier:
              precision    recall  f1-score   support

           0     0.4167    0.1230    0.1899       122
           1     0.1111    0.0090    0.0167       111
           2     0.1905    0.0182    0.0332       220
           3     0.2833    0.1291    0.1774       395
           4     0.5963    0.9205    0.7237      1107

    accuracy                         0.5575      1955
   macro avg     0.3196    0.2400    0.2282      1955
weighted avg     0.4486    0.5575    0.4622      1955

Accuracy on test set: 0.5575447570332481


### SVM with linear kernel

In [19]:
# SVM (linear) Classifier - Hyperparameter Tuning
svm_param_grid = {
    'C': [0.1, 1, 10]
    
}
svm_linear_classifier = SVC(kernel='linear')
svm_grid_search = GridSearchCV(svm_linear_classifier, svm_param_grid, cv=3, n_jobs=-1)
svm_grid_search.fit(x_train_features, y_train)
best_svm_linear_classifier = svm_grid_search.best_estimator_
svm_linear_valid_predictions = best_svm_linear_classifier.predict(x_valid_features)
svm_linear_test_predictions = best_svm_linear_classifier.predict(x_test_features)


In [20]:
print("\nResults on the validation set:")
print("\nSVM (linear) Classifier:")
print(classification_report(y_valid, svm_linear_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, svm_linear_valid_predictions))


Results on the validation set:

SVM (linear) Classifier:
              precision    recall  f1-score   support

           0     0.5750    0.3382    0.4259        68
           1     0.4444    0.0816    0.1379        49
           2     0.3333    0.0952    0.1481       105
           3     0.3900    0.1921    0.2574       203
           4     0.6360    0.9274    0.7545       537

    accuracy                         0.5967       962
   macro avg     0.4758    0.3269    0.3448       962
weighted avg     0.5370    0.5967    0.5288       962

Accuracy: 0.5966735966735967


In [21]:
# Print classification report and accuracy for the SVM on the test set
print("\nResults on the test set:")
print("\nSVM (linear) Classifier:")
print(classification_report(y_test, svm_linear_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, svm_linear_test_predictions))


Results on the test set:

SVM (linear) Classifier:
              precision    recall  f1-score   support

           0     0.5811    0.3525    0.4388       122
           1     0.4375    0.0631    0.1102       111
           2     0.3582    0.1091    0.1672       220
           3     0.3865    0.2025    0.2658       395
           4     0.6518    0.9368    0.7687      1107

    accuracy                         0.6092      1955
   macro avg     0.4830    0.3328    0.3502      1955
weighted avg     0.5486    0.6092    0.5414      1955

Accuracy on test set: 0.6092071611253197


### SVM with rbf kernel

In [22]:

svm_rbf_param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10]
}
svm_rbf_classifier = SVC(kernel='rbf')
svm_rbf_grid_search = GridSearchCV(svm_rbf_classifier, svm_rbf_param_grid, cv=3, n_jobs=-1)
svm_rbf_grid_search.fit(x_train_features, y_train)
best_svm_rbf_classifier = svm_rbf_grid_search.best_estimator_
svm_rbf_valid_predictions = best_svm_rbf_classifier.predict(x_valid_features)
svm_rbf_test_predictions = best_svm_rbf_classifier.predict(x_test_features)


In [23]:
print("\nResults on the validation set:")
print("\nSVM (rbf) Classifier:")
print(classification_report(y_valid, svm_rbf_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, svm_rbf_valid_predictions))


Results on the validation set:

SVM (rbf) Classifier:
              precision    recall  f1-score   support

           0     0.6774    0.3088    0.4242        68
           1     0.5000    0.0816    0.1404        49
           2     0.3214    0.0857    0.1353       105
           3     0.3496    0.2118    0.2638       203
           4     0.6412    0.9218    0.7563       537

    accuracy                         0.5946       962
   macro avg     0.4979    0.3220    0.3440       962
weighted avg     0.5401    0.5946    0.5298       962

Accuracy: 0.5945945945945946


In [24]:
# Print classification report and accuracy for the SVM on the test set
print("\nResults on the test set:")
print("\nSVM (rbf) Classifier:")
print(classification_report(y_test, svm_rbf_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, svm_rbf_test_predictions))


Results on the test set:

SVM (rbf) Classifier:
              precision    recall  f1-score   support

           0     0.6140    0.2869    0.3911       122
           1     0.4737    0.0811    0.1385       111
           2     0.4375    0.1273    0.1972       220
           3     0.3769    0.2557    0.3047       395
           4     0.6606    0.9232    0.7702      1107

    accuracy                         0.6113      1955
   macro avg     0.5125    0.3348    0.3603      1955
weighted avg     0.5647    0.6113    0.5521      1955

Accuracy on test set: 0.6112531969309463


### Logistic Regression

In [25]:
# Logistic Regression Classifier - Hyperparameter Tuning
lr_param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2']
}
lr_classifier = LogisticRegression()
lr_grid_search = GridSearchCV(lr_classifier, lr_param_grid, cv=3, n_jobs=-1)
lr_grid_search.fit(x_train_features, y_train)
best_lr_classifier = lr_grid_search.best_estimator_
lr_valid_predictions = best_lr_classifier.predict(x_valid_features)
lr_test_predictions = best_lr_classifier.predict(x_test_features)



In [26]:
print("\nResults on the validation set:")
print("\nLogistic Regression Classifier:")
print(classification_report(y_valid, lr_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, lr_valid_predictions))



Results on the validation set:

Logistic Regression Classifier:
              precision    recall  f1-score   support

           0     0.6957    0.2353    0.3516        68
           1     0.5000    0.0408    0.0755        49
           2     0.3889    0.0667    0.1138       105
           3     0.3629    0.2217    0.2752       203
           4     0.6330    0.9348    0.7549       537

    accuracy                         0.5946       962
   macro avg     0.5161    0.2999    0.3142       962
weighted avg     0.5470    0.5946    0.5206       962

Accuracy: 0.5945945945945946


In [27]:
# Print classification report and accuracy for the Logistic on the test set
print("\nResults on the test set:")
print("\nLogistic Regression Classifier:")
print(classification_report(y_test, lr_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, lr_test_predictions))


Results on the test set:

Logistic Regression Classifier:
              precision    recall  f1-score   support

           0     0.6538    0.2787    0.3908       122
           1     0.3750    0.0270    0.0504       111
           2     0.4510    0.1045    0.1697       220
           3     0.3506    0.2051    0.2588       395
           4     0.6441    0.9386    0.7640      1107

    accuracy                         0.6036      1955
   macro avg     0.4949    0.3108    0.3267      1955
weighted avg     0.5484    0.6036    0.5312      1955

Accuracy on test set: 0.6035805626598465


### XGBoost

In [29]:
xgb_classifier = XGBClassifier(n_jobs=-1, max_depth=3, learning_rate=0.1, n_estimators=100)
xgb_classifier.fit(x_train_features, y_train)
xgb_valid_predictions = xgb_classifier.predict(x_valid_features)
xgb_test_predictions = xgb_classifier.predict(x_test_features)

In [30]:
print("\nResults on the validation set:")
print("\nXGBoost Classifier:")
print(classification_report(y_valid, xgb_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, xgb_valid_predictions))


Results on the validation set:

XGBoost Classifier:
              precision    recall  f1-score   support

           0     0.4000    0.0588    0.1026        68
           1     1.0000    0.0408    0.0784        49
           2     0.5714    0.0381    0.0714       105
           3     0.4906    0.1281    0.2031       203
           4     0.5854    0.9702    0.7302       537

    accuracy                         0.5790       962
   macro avg     0.6095    0.2472    0.2372       962
weighted avg     0.5719    0.5790    0.4695       962

Accuracy: 0.579002079002079


In [31]:
# Print classification report and accuracy for the XGBoost on the test set
print("\nResults on the test set:")
print("\nXGBoost Classifier:")
print(classification_report(y_test, xgb_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, xgb_test_predictions))


Results on the test set:

XGBoost Classifier:
              precision    recall  f1-score   support

           0     0.8077    0.1721    0.2838       122
           1     0.2500    0.0090    0.0174       111
           2     0.5000    0.0364    0.0678       220
           3     0.4638    0.1620    0.2402       395
           4     0.6059    0.9693    0.7457      1107

    accuracy                         0.5969      1955
   macro avg     0.5255    0.2698    0.2710      1955
weighted avg     0.5576    0.5969    0.4971      1955

Accuracy on test set: 0.5969309462915601


### Random Forest

In [32]:
rf_classifier = RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, min_samples_split=2, min_samples_leaf=1)
rf_classifier.fit(x_train_features, y_train)
rf_valid_predictions = rf_classifier.predict(x_valid_features)
rf_test_predictions = rf_classifier.predict(x_test_features)

In [33]:
print("\nResults on the validation set:")
print("\nRandom Forest Classifier:")
print(classification_report(y_valid, rf_valid_predictions, digits=4))
print("Accuracy:", accuracy_score(y_valid, rf_valid_predictions))


Results on the validation set:

Random Forest Classifier:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        68
           1     0.0000    0.0000    0.0000        49
           2     0.0000    0.0000    0.0000       105
           3     0.0000    0.0000    0.0000       203
           4     0.5582    1.0000    0.7165       537

    accuracy                         0.5582       962
   macro avg     0.1116    0.2000    0.1433       962
weighted avg     0.3116    0.5582    0.3999       962

Accuracy: 0.5582120582120582


In [35]:
# Print classification report and accuracy for the Random Forest on the test set
print("\nResults on the test set:")
print("\nRandom Forest Classifier:")
print(classification_report(y_test, rf_test_predictions, digits=4))
print("Accuracy on test set:", accuracy_score(y_test, rf_test_predictions))


Results on the test set:

Random Forest Classifier:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       122
           1     0.0000    0.0000    0.0000       111
           2     0.0000    0.0000    0.0000       220
           3     0.0000    0.0000    0.0000       395
           4     0.5662    1.0000    0.7231      1107

    accuracy                         0.5662      1955
   macro avg     0.1132    0.2000    0.1446      1955
weighted avg     0.3206    0.5662    0.4094      1955

Accuracy on test set: 0.5662404092071611


### Accuracy of every model

In [36]:
results_dict = {
    "Model": ["Decision Tree", "SVM (linear)", "SVM (rbf)", "Logistic Regression", "XGBoost", "Random Forest"],
    "Accuracy": [accuracy_score(y_test, dt_test_predictions),
                 accuracy_score(y_test, svm_linear_test_predictions),
                 accuracy_score(y_test, svm_rbf_test_predictions),
                 accuracy_score(y_test, lr_test_predictions),
                 accuracy_score(y_test, xgb_test_predictions),
                 accuracy_score(y_test, rf_test_predictions)]
}

# Create a DataFrame from the dictionary
results_df = pd.DataFrame(results_dict)


print("Test Accuracy Results:")
print(results_df)


Test Accuracy Results:
                 Model  Accuracy
0        Decision Tree  0.557545
1         SVM (linear)  0.609207
2            SVM (rbf)  0.611253
3  Logistic Regression  0.603581
4              XGBoost  0.596931
5        Random Forest  0.566240


### Conclusion

I have taken only 5 % of the data in this file, But i tried using 30%, 50% of the data for building the model, and i found there is no difference in the accuracies

And i also found that after using gridsearch the accuracies are decreased

SVM and logistic models are working better than other models